In [21]:
import numpy as np
import pandas as pd
from os import listdir
import reproduccion_ulutas as rp
import reproduccion_ulutas_N_flow as nf
import time

In [22]:
mypath = 'data/instances/A1_A16'
datasets = [f for f in listdir(mypath)] #dataset reading

In [23]:
POPULATION_SIZE = 4   # indicate the initial size of antibodies population
# N is a random % of antibodies with highest sum_intercell_transport selected to be cloned
R = 0.5 # % of best cloned antibodies to the pool of antibodies
B = 0.5 # worst % of the whole population (RECEPTOR EDITING)

MAX_ITERATIONS = 1
MAX_ITERATIONS_WITHOUT_IMPROVEMENT = 1
number_of_runs = 1
log = True # Boolean: print logs at the execution of the algorithm.

In [24]:
#reads data and translates it into part-machine matrix
problem_names, grouping_efficacies, number_of_iterations, elapsed_time = [],[],[],[]
path = mypath + '/'
for dataset in datasets:
    data = path + dataset
    runs = 0
    while runs < number_of_runs: #number of runs for each instance
        matrix, m, p,number_of_operations, columns, rows = rp.part_machine_incidence_matrix(data)
        if log == True: print("\nPart-machine matrix\n",matrix)
        #GENERATION OF INITIAL POPULATION
        antibodies = rp.generation_initial_population(p = p, m= m, population_size = POPULATION_SIZE)
        if log == True: print("\nInitial antibodies\n",antibodies)
        iterations, best_solution, improvement_counter = 0, 100, 0
        start = time.time() #start time for each problem

        while iterations <= MAX_ITERATIONS:
            if log == True: print("\n Iteration number",iterations+1)
            #identifies cells in the antibodies
            total_cells = rp.cell_identification(antibodies = antibodies)
            if log == True: print("\n Cells\n", total_cells)
            #translates numbers into parts and machines
            total_machines, total_parts = rp.decode_cells(total_cells = total_cells, rows= rows, columns = columns, p = p, log = log) 

            #calculates part-machine matrix for each antibody
            antibody_matrices = rp.create_machine_part_matrix(matrix = matrix, 
                                                            antibodies = antibodies, 
                                                            total_machines = total_machines, 
                                                            total_parts = total_parts)
            #evaluates efficacie of each antibody
            efficacies, voids, exceptions = nf.evaluate_antibodies(antibody_matrices = antibody_matrices, 
                                                                                total_cells = total_cells,
                                                                                number_of_operations = number_of_operations)
            if log == True: print("\n Efficacies",efficacies)
            #evaluates intercell transports of each antibody
            intercell_transports, sum_intercell_transport = nf.evaluate_intercell_transport(antibody_matrices = antibody_matrices,
                                                                                            total_cells = total_cells,
                                                                                            rows = rows)
            if log == True: print("\n Intercell Transports",intercell_transports)
            if log == True: print("\n Sum of Intercell Transports",sum_intercell_transport)
            # SELECTS N% OF ANTIBODIES WITH HIGHES ANTIBODIES (SELECTION ROULETTE WHEEL)
            N = np.random.rand(1) # N is a random number each iteration
            if log: print("N% =",N[0])
            cloned_antibodies, positions_antibodies_selected = nf.antibodies_selection_v3(antibodies=antibodies, N=N[0], sum_intercell_transport = sum_intercell_transport)
            if log == True: print("\n Cloned antibodies\n",cloned_antibodies)
            if log == True: print("\n Cloned antibodies positions (pool)\n",positions_antibodies_selected)
            # MUTATION
            rp.mutate_cloned_antibodies(cloned_antibodies = cloned_antibodies, log= log)
            if log == True: print("\n Mutation of cloned antibodies\n",cloned_antibodies)
            #CLONES: identifies cells in the antibody
            cloned_total_cells = rp.cell_identification(cloned_antibodies)
            #CLONES: translates numbers into parts and machines
            cloned_total_machines, cloned_total_parts = rp.decode_cells(total_cells = cloned_total_cells,rows= rows, columns = columns, p = p, log = log)
            #CLONES: calculates part-machine matrix for each antibody
            cloned_antibody_matrices = rp.create_machine_part_matrix(matrix = matrix,
                                                                        antibodies = cloned_antibodies, 
                                                                        total_machines = cloned_total_machines, 
                                                                        total_parts = cloned_total_parts)                                                          
            #CLONES evaluates efficacie of each cloned antibody
            cloned_efficacies, cloned_voids, cloned_exceptions = nf.evaluate_antibodies(cloned_antibody_matrices,
                                                                                        cloned_total_cells,
                                                                                        number_of_operations)
            if log == True: print("\n Cloned Efficacies",cloned_efficacies)
            #CLONES evaluates intercell transports of each cloned antibody
            cloned_intercell_transports, cloned_sum_intercell_transport = nf.evaluate_intercell_transport(antibody_matrices = antibody_matrices,
                                                                                            total_cells = total_cells,
                                                                                            rows = rows)
            if log == True: print("\n Cloned Intercell Transports",cloned_intercell_transports)
            if log == True: print("\n Cloned Sum of Intercell Transports",cloned_sum_intercell_transport)
            # Add R% of best cloned antibodies to the pool of antibodies
            antibodies, efficacies, sum_intercell_transport = nf.select_best_cloned_antibodies_v2(antibodies = antibodies,
                                                                            cloned_antibodies = cloned_antibodies,
                                                                            efficacies = efficacies, 
                                                                            cloned_efficacies = cloned_efficacies,
                                                                            sum_intercell_transport = sum_intercell_transport, 
                                                                            cloned_sum_intercell_transport = cloned_sum_intercell_transport,
                                                                            R = R,
                                                                            positions_antibodies_selected = positions_antibodies_selected, 
                                                                            log = log)
            if log == True: print("\n New antibodies pool \n",antibodies)                                                                        
            # RECEPTOR EDITING: Remove worst members of the antibodies pool
            antibodies, efficacies, sum_intercell_transport, amount_antibodies_erased = nf.receptor_editing(antibodies_pool = antibodies, 
                                                                                        efficacies = efficacies, 
                                                                                        sum_intercell_transport=sum_intercell_transport, 
                                                                                        B = B, 
                                                                                        log = log)
            if log == True: print("\n Antibodies pool after removal \n", antibodies)
            #GENERATION OF NEW RANDOM ANTIBODIES
            number_new_random_antibodies = amount_antibodies_erased
            new_random_antibodies = rp.generation_initial_population(p = p, 
                                                                m = m, 
                                                                population_size = number_new_random_antibodies)
            if log == True: print("\n New random antibodies \n",new_random_antibodies)                                                        
            antibodies = np.concatenate((antibodies, new_random_antibodies), axis = 0)
            if log == True: print("\n Antibodies pool after adding new random antibodies \n",antibodies)
            
            #TERMINATION CRITERIA AND SELECTION OF BEST SOLUTION
            iteration_best_solution = np.amax(sum_intercell_transport) #max value efficacy for this iteration
            index_best_solution = np.argmax(sum_intercell_transport)
            best_solution_efficacy = efficacies[index_best_solution]
            best_antibody = antibodies[index_best_solution]
            if log: print("\n Best solution at the moment is {} (antibody {}, index {})".format(iteration_best_solution,best_antibody,index_best_solution))
            if iteration_best_solution >= best_solution:
                improvement_counter = improvement_counter + 1
                if improvement_counter >= MAX_ITERATIONS_WITHOUT_IMPROVEMENT:
                    if log: print("Maximun number of iterations without improvement reached: {}".format(MAX_ITERATIONS_WITHOUT_IMPROVEMENT))
                    if log: print("Best solution obtained is {}".format(best_solution))
                    break
            else:
                best_solution = iteration_best_solution
                improvement_counter = 0
            #update iteration counter
            iterations = iterations + 1
        runs = runs + 1
        end = time.time()
        #saving results:
        problem_names.append(dataset)
        grouping_efficacies.append(iteration_best_solution)
        number_of_iterations.append(iterations)
        elapsed_time.append(end-start)
        print(dataset,"\tEficcacy: ", best_solution_efficacy, "\tTransports: ",iteration_best_solution, "\tNumber of iterations:", iterations, "\tElapsed time", end-start)
        
results = {"Problem": problem_names,
            "Efficacy": grouping_efficacies,
            "Iterations": number_of_iterations,
            "Elapsed Time": elapsed_time}
results = pd.DataFrame(results)
results.to_csv('resultsN_flow.csv')


Part-machine matrix
     M1  M2  M3  M4  M5
P1   0   1   1   0   1
P2   1   0   0   1   0
P3   0   1   1   0   0
P4   1   0   0   1   0
P5   1   0   0   0   0
P6   1   0   0   1   0
P7   0   0   1   0   1

Initial antibodies
 [[ 5  0  3  9 12  6  4  2  0 11  1  0  0  7 10  8]
 [ 4 12  1  6  0  9  3  8  0  0 10  7  2  0 11  5]
 [ 4 12  6  0  0  1  7 10  3  8 11  0  2  0  9  5]
 [ 8  0  4  0  2  0 11  0 10  6  7 12  3  9  5  1]]

 Iteration number 1

 Cells
 [[[5], [3, 9, 12, 6, 4, 2], [11, 1], [7, 10, 8]], [[4, 12, 1, 6], [9, 3, 8], [10, 7, 2], [11, 5]], [[4, 12, 6], [1, 7, 10, 3, 8, 11], [2], [9, 5]], [[8], [4], [2], [11], [10, 6, 7, 12, 3, 9, 5, 1]]]
decoded antibodies [['P5'], ['P3', 'M2', 'M5', 'P6', 'P4', 'P2'], ['M4', 'P1'], ['P7', 'M3', 'M1']]
decoded antibodies [['P4', 'M5', 'P1', 'P6'], ['M2', 'P3', 'M1'], ['M3', 'P7', 'P2'], ['M4', 'P5']]
decoded antibodies [['P4', 'M5', 'P6'], ['P1', 'P7', 'M3', 'P3', 'M1', 'M4'], ['P2'], ['M2', 'P5']]
decoded antibodies [['M1'], ['P4'], ['P

In [25]:
best_solution

10

In [26]:
iteration_best_solution

10

In [27]:
best_antibody

array([ 8,  0,  4,  7,  2,  0, 11,  0, 10,  6,  0, 12,  3,  9,  5,  1])

In [28]:
cell_identified_best_antibody = rp.cell_identification([best_antibody])
cell_identified_best_antibody

[[[8], [4, 7, 2], [11], [10, 6], [12, 3, 9, 5, 1]]]

In [29]:
total_machines, total_parts = rp.decode_cells(total_cells = cell_identified_best_antibody, rows= rows, columns = columns, p = p, log = log)

antibody_matrices = rp.create_machine_part_matrix(matrix=matrix,
                                                antibodies=[best_antibody], 
                                                total_machines=total_machines, 
                                                total_parts=total_parts)
antibody_matrices    

decoded antibodies [['M1'], ['P4', 'P7', 'P2'], ['M4'], ['M3', 'P6'], ['M5', 'P3', 'M2', 'P5', 'P1']]


[    M1  M4  M3  M5  M2
 P4   1   1   0   0   0
 P7   0   0   1   1   0
 P2   1   1   0   0   0
 P6   1   1   0   0   0
 P3   0   0   1   0   1
 P5   1   0   0   0   0
 P1   0   0   1   1   1]

In [30]:
efficacies, affinities, voids, exceptions = rp.evaluate_antibodies(antibody_matrices = antibody_matrices, 
                                                                    total_cells=cell_identified_best_antibody,
                                                                    number_of_operations = number_of_operations)
efficacies

[0.0]